# Introduction to PDEpy
This is a numerical library for solving PDEs with finite difference methods.
## Supported PDEs
### 2D Linear (Constant/Variable Coefficient) PDEs on Regular/Irregular Domains with Dirichlet Boundary Condition
Supported operators: a linear combination of $\{\mathit{I}, \frac{\partial}{\partial x}, \frac{\partial}{\partial y}, \frac{\partial^2}{\partial x^2}, \frac{\partial^2}{\partial y^2}, \nabla^2, \dots\}$  
Default rate of convergence: second-order of accuracy.  
Documentations: https://github.com/Walden-Shen/pdepy/blob/master/docs/2dPDEs.ipynb

### Linear (Constant/Variable coefficient) Time-dependent PDEs
#### 1D Problems
Supported operators: $\{\mathit{I}, \frac{\partial}{\partial t}, \frac{\partial}{\partial x}, \frac{\partial^2}{\partial x^2}, \dots\}$  
Default algorithm: Crank-Nicolson method (unconditionally stable).  
Documentations: https://github.com/Walden-Shen/pdepy/blob/master/docs/time_dependent_PDEs.ipynb

#### 2D Problems on Regular Domains
Supported operators: $\{\mathit{I}, \frac{\partial}{\partial t}, \frac{\partial}{\partial x}, \frac{\partial}{\partial y}, \frac{\partial^2}{\partial x^2}, \frac{\partial^2}{\partial y^2}, \dots\}$  
Default algorithm: the explicit method (conditionally stable).  
Documentations: https://github.com/Walden-Shen/pdepy/blob/master/docs/time_dependent_PDEs.ipynb

# Gist of This Library
- User Friendly  
Whatever type of PDE the user wants to solve, all he/she needs to do is to set the boundary condition, plug the PDE into the solver, and finally call the function **solve()**. No need to memorize tons of APIs, since the solver will automatically detect the type of the PDE, whether its domain is regular, etc., and select the appropriate algorithm for you.  
     
     
- Extensible & Easy to Extend  
It's impossible that the library will always meet all needs from its users, e.g., higher-order differential operators, more accurate approximation, etc. So when I was designing this library, I made it extremely easy to add new operators, or to modify the existing numerical stencils.  

# Code Structure

## src.util.diff_operators
There are two child packages __core__ and __impl__ inside it. **diff_operators.core** contains the most generic operators:
```python
class diff_operator(object):
    def __init__(self, stencil, coefficient = lambda x, y: 1, is_time_dependent = False):
        # a list of tuple whose first element is relative position of node in the stencil, 
        # second element is the coefficient in the fdm equation
        self.stencil = stencil
        # the coefficient before this operator, e.g., lambda x, y: 1/delta_x^2
        self.coefficient = coefficient
        self.is_time_dependent = is_time_dependent
    def getIrregularStencil(self, dx, tau, direction):
        raise NotImplementedError
```
and
```python
class time_dependent_operator(diff_op.diff_operator):
    def __init__(self, implicit_stencil, explicit_stencil = None, coefficient = lambda x, y: 1):
        super().__init__(implicit_stencil, coefficient, True)
        self.explicit_stencil = explicit_stencil 
        # for 2d time-dependent equation. The first argument in tuple represents x coordinate offset,
        # while the second argument represents y coordinate offset
    def get_implicit_stencil(self):
        if self.stencil is None: raise NotImplementedError
        return self.stencil
    def get_explicit_stencil(self):
        if self.explicit_stencil is None: raise NotImplementedError
        return self.explicit_stencil
```
It's intuitive that the time-dependent operator class is a subclass of usual operator class. And I made the type of coefficient be a lambda function, which **allows the coefficient before the operator to be either a function g(x, y) or a constant**.    
  
Then let's take a look at concrete operators in **diff_operators.impl**.
```python
class ddx(diff_op.diff_operator):
    def __init__(self, dx, coefficient = lambda x, y: 1):
        super().__init__([((-1, 0), -1/(2*dx)), ((1, 0), 1/(2*dx))], coefficient)
```
The first argument $[((-1, 0), -1/(2*dx)), ((1, 0), 1/(2*dx))]$ represents the stencil we used to approximate $\frac{du}{dx}$, which is $\frac{u(x+\Delta x, y)-u(x-\Delta x, y)}{2\Delta x}\sim \frac{du}{dx} + \mathcal{O}(\Delta x^2)$.   
From this example, we can see that $(-1, 0)$ represents $(x-\Delta x, y)$ and $-1/(2*dx)$ represents $-\frac{1}{2\Delta x}$.   
Also note that the lambda function **coefficient** means the coefficient before this operator, e.g., if we have $-sin(x)sin(y)\frac{du}{dx}$, the **coefficient** would be __lambda x, y: -math.sin(x)\*math.sin(y)__.  
  
Here is a time-dependent operator example. The implicit stencil is for Crank-Nicolson method, while the explicit stencil is for the explicit method. User can add new time-dependent operators in the manner of this class.
```python
class td_ddx(time_dependent_op.time_dependent_operator):
    def __init__(self, dx, coefficient = lambda x, y: 1):
        implicit_stencil = [((-1, 0), -1/(4*dx)), ((1, 0), 1/(4*dx)), ((-1, -1), -1/(4*dx)), ((1, -1), 1/(4*dx))] 
        # the tuple in implicit_stencil is interpreted as (x_offset, t_offset)
        explicit_stencil = [((-1, 0, -1), -1/(2*dx)), ((1, 0, -1), 1/(2*dx))] 
        # the tuple in explicit_stencil is interpreted as (x_offset, y_offset, t_offset)
        super().__init__(implicit_stencil, explicit_stencil=explicit_stencil, coefficient=coefficient)
```
## src.util.diff_op_expression
This is a class for storing all the operators in a PDE. It can be seen as a list that is able to classify the inputed PDE. 
## src.util.domain_conditions
This is one of the boundary condition that users have to fill in. The arguments' names are self-explanatory:
- **inDomain**: Return whether the given point belongs to the domain.
- **onBoundary**: Return whether the given point belongs to the boundary of the domain.
- **getBoundaryValue**: Return the boundary value given a boundary point.
- **domain**: An object instantiated from the Domain class. It represents a rectangle domain that includes the real domain of the PDE (which can be irregular).
- **getNearestPoint**: A tuple contains two lambda functions that takes a point (x, y) as input. They return the horizontally closest boundary point, and the vertically closest boundary point respectively. Only used when the PDE problem is on an irregular domain.  
  
```python
class dirichlet_bc(bc.boundary_condition):
    def __init__(self, inDomain, onBoundary, getBoundaryValue, domain, getNearestPoint = (None, None)):
        super().__init__(inDomain, onBoundary, getBoundaryValue, getNearestPoint)
        self.domain = domain # this is the class domain
```
  
## src.fdm.solver
This package contains the algorithm for our solver.  
```python
def solve(self, nx, ny = 1, nt = None):
        if self.diff_op_expression.is_time_dependent():
            if not self.diff_op_expression.all_ops_are_time_dependent(): raise TypeError
            if nt is None: return self.time_dependent_implicit_solve(nx, ny)
            return self.time_dependent_explicit_solve(nx, ny, nt)
        else:
            return self.spatial_solve(nx, ny)
```
This function is the entry point of our solver. It calls different solving algorithms based on the type of the PDE problem. 
- **spatial_solve()** contains the algorithm for solving 2D PDEs on the regular/irregular domain. 
- **time_dependent_implicit_solve()** is a function for solving 1D time-dependent PDEs with Crank-Nicolson method. 
- **time_dependent_explicit_solve()** solves 2D time-dependent PDEs with the explicit method.  

